In [ ]:
import tensorflow as tf
from tensorflow import keras

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())

In [ ]:
import numpy as np
import pandas as pd
import os
import PIL.Image
from PIL.Image import Resampling
PIL.Image.MAX_IMAGE_PIXELS = None

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from keras import layers
from tensorflow.keras import losses

Загрузка данных и разбиение на выборки

In [ ]:
path_labels = np.load('C:/Users/USRx/Desktop/Daria/category_img_new.npy',allow_pickle=True)

In [ ]:
df = pd.DataFrame(path_labels, columns=['path','label'])

In [ ]:
df['label'] = df['label'].astype(str)

In [ ]:
df.head()

In [ ]:
train_df = pd.DataFrame(columns=['path', 'label'])
val_df = pd.DataFrame(columns=['path', 'label'])
test_df = pd.DataFrame(columns=['path', 'label'])

unique_labels = df['label'].unique()

In [ ]:
for label in unique_labels:
    label_data = df[df['label'] == label]
    
    train_size = int(len(label_data)*0.8)
    val_size = int(len(label_data) * 0.1)
    
    train_data = label_data[:train_size]
    val_data = label_data[train_size:train_size+val_size]
    test_data = label_data[train_size+val_size:]
    
    train_df = pd.concat([train_df, train_data])
    val_df = pd.concat([val_df, val_data])
    test_df = pd.concat([test_df, test_data])

In [ ]:
print("Train Data:")
print(train_df.shape,len(train_df['label'].unique()))
print("nTest Data:")
print(val_df.shape,len(test_df['label'].unique()))
print("nTest Data:")
print(test_df.shape,len(test_df['label'].unique()))

In [ ]:
train_df = train_df.sample(frac=1).reset_index(drop=True)
val_df = val_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [ ]:
print(train_df.shape[0],val_df.shape[0],test_df.shape[0])

Преобученные модели

In [ ]:
from keras_applications.vgg16 import VGG16, preprocess_input
from keras_applications.mobilenet_v2 import MobileNetV2
from keras_applications.resnet_v2 import ResNet50V2
from tensorflow.keras.optimizers import Adam

In [ ]:
conv_base = keras.applications.resnet_v2.ResNet50V2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
conv_base.trainable = False

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None, 
    x_col='path', 
    y_col='label', 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=None, 
    x_col='path', 
    y_col='label', 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None, 
    x_col='path', 
    y_col='label', 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))
x = conv_base(inputs)
x = layers.Flatten()(x)
outputs = layers.Dense(46, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="categorical_crossentropy",
    optimizer=Adam(),
    metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
    filepath="resnet50v2.keras",
    save_best_only=True,
    monitor="val_accuracy")
]

In [ ]:
history = model.fit(
    train_generator,
    epochs=35,
    validation_data=val_generator,
    callbacks=callbacks)